In [1]:
################################################
################## Biblioteki ##################
################################################

import numpy as np
import cv2
import matplotlib
import copy
import sys
import math
import time
from queue import *
from tqdm import tnrange, tqdm_notebook
from matplotlib import pyplot as plt
from numpy import linalg as LA

In [2]:
###############################################
####### Zabezpieczenie poziomu jasności #######
###############################################

def min_max(i):
    return max(0, min(i, 255))

In [3]:
###############################################
############# Filtracja medianowa #############
###############################################

def median_filter(img, function, n = 3, colors = [0, 1, 2]):
    result = copy.deepcopy(img)
    for i in range(int(n/2), len(result) - int(n/2)):
        for j in range(int(n/2), len(result[i]) - int(n/2)):
            for c in colors:
                result[i, j, c] = function(img[i-int(n/2):i+int(n/2)+1:, j-int(n/2):j+int(n/2)+1:, c])
    return result

In [4]:
################################################
######## Operacje zamknięcia i otwarcia ########
################################################

def close_img(img, n = 3, colors = [0, 1, 2]):
    return median_filter(median_filter(img, np.max, n, colors), np.min, n, colors)

def open_img(img, n = 3, colors = [0, 1, 2]):
    return median_filter(median_filter(img, np.min, n, colors), np.max, n, colors)

def prepare_segments_open_close(img, n = 3, colors = [0, 1, 2]):
    return open_img(close_img(img, n, colors), n, colors)

def prepare_segments_close_open(img, n = 3, colors = [0, 1, 2]):
    return close_img(open_img(img, n, colors), n, colors)

In [5]:
################################################
######## Zastosowanie filtru splotowego ########
################################################

def splot_filter(img, filter_matrix, colors = [0, 1, 2]):
    result = copy.deepcopy(img)

    size = int(len(filter_matrix[0])/2)
    
    for i in range(size, len(result) - size):
        for j in range(size, len(result[i]) - size):
            for c in [0, 1, 2]:
                result[i, j, c] = min_max(np.sum(
                    img[i-size:i + size + 1:, j-size:j + size + 1:, c] * filter_matrix)
                                         )   

In [6]:
################################################
############ Filtr górnoprzepustowy ############
################################################

def filter_hight_pass(img, colors = [0, 1, 2]):
    filter_matrix = np.array([[0, -1, 0], 
                            [-1, 5, -1], 
                            [0, -1, 0]])
    
    return splot_filter(img, filter_matrix, colors)
    

In [7]:
################################################
################## Progowanie ##################
################################################

def thresholding(img, color, first_threshold, second_threshold = 256, difference = 0):
    result = copy.deepcopy(img)
    colors = np.array([0, 1, 2])
    mask = np.ones(colors.shape, dtype=bool)  
    mask[color] = 0
        
    for i in range(len(result)):
        for j in range(len(result[i])):
            if result[i, j, color] > first_threshold and (result[i, j, colors[mask]] < second_threshold).all() and (result[i, j, colors[mask]] + difference < result[i, j, color]).all():
                result[i, j, color] = 255
            else:
                result[i, j, color] = 0

    return np.array(result).astype('uint8')

In [8]:
###############################################
######### Wyświetlanei jednego koloru #########
###############################################

def reduce_color_to_one(img, color):
    return np.array([[[img[i, j, color]] for j in range(len(img[i]))] for i in range(len(img))])

In [9]:
##############################################
############ Wykrywanie segmentów ############
##############################################

def find_segment(i, j, segment_index, segments_matrix, segments_descriptors, segmentation_image, min_area):
    if(segments_matrix[i, j] == -1 and segmentation_image[i, j, 0] == 255):
        min_i = 9999999
        max_i = 0
        min_j = 9999999
        max_j = 0
        area = 0
        
        q = Queue()
        q.put((i,j), False)
        while not q.empty():
            point_i, point_j = q.get()
            if not (point_i < 0 or point_j < 0 or point_i > len(segments_matrix) - 1 or point_j > len(segments_matrix[0]) - 1):
                if segments_matrix[point_i, point_j] == -1 and segmentation_image[point_i, point_j, 0] == 255:
                    segments_matrix[point_i, point_j] = segment_index
                    area = area + 1
                    if min_i > point_i:
                        min_i = point_i
                    if max_i < point_i:
                        max_i = point_i
                    if min_j > point_j:
                        min_j = point_j
                    if max_j < point_j:
                        max_j = point_j
                    
                    q.put((point_i-1, point_j-1), False)
                    q.put((point_i-1, point_j), False)
                    q.put((point_i-1, point_j+1), False)
                    q.put((point_i, point_j-1), False)
                    q.put((point_i, point_j+1), False)
                    q.put((point_i+1, point_j-1), False)
                    q.put((point_i+1, point_j), False)
                    q.put((point_i+1, point_j+1), False)
        
        if area < min_area:
            segments_matrix[segments_matrix==segment_index] = -2
            return False, []
            
        return True, [[min_i, max_i, min_j, max_j], [area, 0], []]
    else:
        return False, []

In [10]:
##############################################
############ Wykrywanie segmentów ############
##############################################

def prepare_segments_matrix(segmentation_image, min_area):
    segments_matrix = np.array([[-1 for _ in range(len(segmentation_image[0]))] for _ in range(len(segmentation_image))])
    segments_descriptors = []
    segments_count = 0
    for i in tnrange(len(segmentation_image), desc = 'Identyfikowanie segmentów'):
        for j in range(len(segmentation_image[i])):
            if(segments_matrix[i, j] == -1):
                succes, descriptor = find_segment(i, j, segments_count, segments_matrix, segments_descriptors, segmentation_image, min_area)
                if succes:
                    segments_descriptors.append(descriptor)
                    segments_count = segments_count + 1
    
    segments_count = segments_count - 1
    return segments_matrix, segments_descriptors, segments_count

In [11]:
###################################################
################## Liczenie cech ##################
###################################################

In [12]:
###################################################
###################### Obwód ######################
###################################################

def calculate_circuit(segments_matrix, segments_descriptors, index):
    result = 0
    max_i = len(segments_matrix) - 1
    max_j = len(segments_matrix[0]) - 1
    for i in range(segments_descriptors[index][0][0], segments_descriptors[index][0][1] + 1):
        for j in range(segments_descriptors[index][0][2], segments_descriptors[index][0][3] + 1):
            if (segments_matrix[i, j] == index and (i == 0 or i == max_i or j == 0 or j == max_j)):
                result = result + 1
            elif (segments_matrix[i, j] == index 
                and (
                    segments_matrix[i - 1, j - 1] != index or
                    segments_matrix[i - 1, j] != index or
                    segments_matrix[i - 1, j + 1] != index or
                    segments_matrix[i, j - 1] != index or
                    segments_matrix[i, j + 1] != index or
                    segments_matrix[i + 1, j - 1] != index or
                    segments_matrix[i + 1, j] != index or
                    segments_matrix[i + 1, j + 1] != index
                    )
               ):
                result = result + 1
    return result

In [13]:
###################################################
################### Malinowska ####################
###################################################

def calculate_malinowska(S, L):
    return (L/(2*math.sqrt(S*math.pi))) - 1

In [14]:
###################################################
##################### Moment ######################
###################################################

def calculate_moment(p, q, segments_matrix, segments_descriptors, index):
    result = 0
    for i in range(segments_descriptors[index][0][1] + 1 - segments_descriptors[index][0][0]):
        for j in range(segments_descriptors[index][0][3] + 1 - segments_descriptors[index][0][2]):
            if segments_matrix[i + segments_descriptors[index][0][0], j + segments_descriptors[index][0][2]] == index:
                result = result + pow(i, p) * pow(j, q)
                
    return result

In [15]:
###################################################
################# Centrum obrazu ##################
###################################################

def find_center(segments_matrix, segments_descriptors, index):
    m10 = calculate_moment(1, 0, segments_matrix, segments_descriptors, index);
    m01 = calculate_moment(0, 1, segments_matrix, segments_descriptors, index);
    m00 = calculate_moment(0, 0, segments_matrix, segments_descriptors, index);

    return (m10/m00, m01/m00)

In [16]:
###################################################
################ Moment centralny #################
###################################################

def calculate_central_moment(p, q, segments_matrix, segments_descriptors, index, center_i = -1, center_j = -1):
    result = 0
    
    if center_i == -1 or center_j == -1:
        center_i, center_j = find_center(segments_matrix, segments_descriptors, index)
        
    for i in range(segments_descriptors[index][0][1] + 1 - segments_descriptors[index][0][0]):
        for j in range(segments_descriptors[index][0][3] + 1 - segments_descriptors[index][0][2]):
            if segments_matrix[i + segments_descriptors[index][0][0], j + segments_descriptors[index][0][2]] == index:
                result = result + pow(i - center_i, p) * pow(j - center_j, q)
                
    return result

In [17]:
###################################################
########### Niezmienniki geometryczne #############
###################################################

def calculate_invariants(segments_matrix, segments_descriptors, index):
    results = [0] * 11
    
    m00 = calculate_moment(0, 0, segments_matrix, segments_descriptors, index);
    center_i, center_j = find_center(segments_matrix, segments_descriptors, index)
    M = np.zeros((4, 4))
    for i in range(4):
        for j in range(4):
            M[i, j] = calculate_central_moment(i, j, segments_matrix, segments_descriptors, index, center_i, center_j)
    
    #M1
    results[1] = (M[2, 0] + M[0, 2]) / pow(m00, 2)
    #M2
    results[2] = (pow(M[2, 0] + M[0, 2], 2) + 4 * pow(M[1, 1], 2)) / pow(m00, 4)
    #M3
    results[3] = (pow(M[3, 0] - 3 * M[1, 2], 2) + pow(3 * M[2, 1] - M[0, 3], 2)) / pow(m00, 5)
    #M4
    results[4] = (pow(M[3, 0] + M[1, 2], 2) + pow(M[2, 1] + M[0, 3], 2)) / pow(m00, 5)    
    #M5
    results[5] = ((M[3, 0]- 3 * M[1, 2]) * (M[3, 0] + M[1, 2]) * 
                  (pow(M[3, 0]+ M[1, 2], 2) - 3 * pow(M[2, 1] + M[0, 3], 2)) +
                 (3 * M[2, 1] - M[0, 3]) * (M[2, 1] + M[0, 3]) * 
                  (3 * pow(M[3, 0] + M[1, 2], 2) - pow(M[2, 1] + M[0, 3], 2))
                 ) / pow(m00, 10)
    #M6
    results[6] = ((M[2, 0] - M[0, 2])*(pow(M[3, 0] + M[1, 2], 2) - pow(M[2, 1] + M[0, 3], 2)) +
                 4 * M[1, 1] * (M[3, 0] + M[1, 2]) * (M[2, 1] + M[0, 3])) / pow(m00, 7)
    #M7
    results[7] = (M[2, 0] * M[0, 2] - pow(M[1, 1], 2)) / pow(m00, 4)
    #M8
    results[8] = (M[3, 0] * M[1, 2] +  M[2, 1] * M[0, 3] - pow(M[1, 2], 2) - pow(M[2, 1], 2)) / pow(m00, 5)    
    #M9
    results[9] = (M[2, 0] * (M[2, 1] * M[0, 3] - pow(M[1, 2], 2)) + 
                 M[0, 2] * (M[0, 3] * M[1, 2] - pow(M[2, 1], 2)) -
                 M[1, 1] * (M[3, 0] * M[0, 3] - M[2, 1] * M[1, 2])) / pow(m00, 7)
    #M10
    results[10] = (pow(M[3, 0] * M[0, 3] - M[1, 2] * M[2, 1], 2) - 
                  4*(M[3, 0]*M[1, 2] - pow(M[2, 1], 2))*(M[0, 3] * M[2, 1] - M[1, 2])) / pow(m00, 10)
    
    return results

In [18]:
###################################################
########### Wykryj czerwoną strzałkę ##############
###################################################

def detect_red_arrow(img):
    color = 2
    treshold_1 = 40
    treshold_2 = 170
    difference = 35
    median_range = 5
    min_area = 100
    print('\nKolor czerwony')
    print('\rFiltrowanie medianowe', end="")
    img_after_filtering = prepare_segments_open_close(img, 5, [color])
    print('\rProgowanie                ', end="")
    img_after_tresholding = thresholding(img_after_filtering, color, treshold_1, treshold_2, difference)
    print('\rFiltrowanie medianowe'        , end="")
    img_after_filtering_2 = close_img(open_img(img_after_tresholding, 3, [color]), 3, [color])
    print('\rRedukcja do jednego koloru', end="")
    segmentation_image = reduce_color_to_one(img_after_filtering_2, color)
    print('\rIdentyfikacja segmentów                                ', end="")
    segments_matrix, segments_descriptors, segments_count = prepare_segments_matrix(segmentation_image, min_area)
    print('\rObliczenie cech                                ', end="")
    for i in tnrange(segments_count + 1, desc = 'Obliczenie wartości cech'):
        segments_descriptors[i][1][1] = calculate_circuit(segments_matrix, segments_descriptors, i)

        segments_descriptors[i][2] = calculate_invariants(segments_matrix, segments_descriptors, i)

        segments_descriptors[i][2][0] = calculate_malinowska(segments_descriptors[i][1][0], segments_descriptors[i][1][1])   
    detected = []
    print('\rAnaliza cech', end="")
    for i in tnrange(len(segments_descriptors), desc = 'Sprawdzenie segmentów'):
        if (sum((
            0.4 <= segments_descriptors[i][2][0] <= 3, 
            0.10 <= segments_descriptors[i][2][1] <= 0.6,
            0.01 <= segments_descriptors[i][2][2] <= 0.5,
            0.0001 <= segments_descriptors[i][2][3] <= 0.07,
            0.0001 <= segments_descriptors[i][2][4] <= 0.03,
            -3.5e-05 <= segments_descriptors[i][2][5] <= 3.0e-04,
            -10.0e-04 <= segments_descriptors[i][2][6] <= 4.0e-03,
            0.001 <= segments_descriptors[i][2][7] <= 0.5,
            -6.0e-03 <= segments_descriptors[i][2][8] <= -9e-05,
            -6.0e-04 <= segments_descriptors[i][2][9] <= 9.5e-05,
            -8e-06 <= segments_descriptors[i][2][10] <= 2.0e-05
        )) >= 11):
            detected.append(i)
                
    return detected, segments_descriptors, segments_matrix

In [19]:
###################################################
########### Wykryj niebieską strzałkę #############
###################################################

def detect_blue_arrow(img):
    color = 0
    treshold_1 = 45
    treshold_2 = 155
    difference = 0
    median_range = 3
    min_area = 100
    print('\nKolor niebieski')
    print('\rProgowanie', end="")
    img_after_tresholding = thresholding(img, color, treshold_1, treshold_2, difference)
    print('\rFiltrowanie medianowe', end="")
    img_after_filtering = prepare_segments_close_open(img_after_tresholding, median_range, [color])
    print('\rRedukcja do jednego koloru', end="")
    segmentation_image = reduce_color_to_one(img_after_filtering, color)
    print('\rIdentyfikacja segmentów                                ', end="")    
    segments_matrix, segments_descriptors, segments_count = prepare_segments_matrix(segmentation_image, min_area)
    print('\rObliczenie cech                        ', end="")    
    for i in tnrange(segments_count + 1, desc = 'Obliczenie wartości cech'):
        segments_descriptors[i][1][1] = calculate_circuit(segments_matrix, segments_descriptors, i)

        segments_descriptors[i][2] = calculate_invariants(segments_matrix, segments_descriptors, i)

        segments_descriptors[i][2][0] = calculate_malinowska(segments_descriptors[i][1][0], segments_descriptors[i][1][1])
    detected = []
    print('\rAnaliza cech', end="")
    for i in tnrange(len(segments_descriptors), desc = 'Sprawdzenie segmentów'):
        if (sum((
            0.50 <= segments_descriptors[i][2][0] <= 2.0, 
            0.15 <= segments_descriptors[i][2][1] <= 0.4,
            0.01 <= segments_descriptors[i][2][2] <= 0.15,
            0.0003 <= segments_descriptors[i][2][3] <= 0.006,
            8.0e-06 <= segments_descriptors[i][2][4] <= 9.0e-04,
            -10.0e-08 <= segments_descriptors[i][2][5] <= 2.0e-06,
            -3.0e-05 <= segments_descriptors[i][2][6] <= 2.0e-04,
            0.006 <= segments_descriptors[i][2][7] <= 0.05,
            -8.0e-04 <= segments_descriptors[i][2][8] <= -4.0e-05,
            -7.0e-05 <= segments_descriptors[i][2][9] <= -2.0e-06,
            -3.5e-07 <= segments_descriptors[i][2][10] <= 0
        )) >= 11):
            detected.append(i)
    return detected, segments_descriptors, segments_matrix

In [20]:
#####################################################
###### Czy potencjalne strzałki tworzą logo #########
#####################################################

def is_connected(blue_arrow, red_arrow, segments_descriptors_blue, segments_descriptors_red, segments_matrix_blue, segments_matrix_red):
    if (segments_descriptors_blue[blue_arrow][1][0] < segments_descriptors_red[red_arrow][1][0] 
        or 0.1 * segments_descriptors_blue[blue_arrow][1][0] > segments_descriptors_red[red_arrow][1][0]):
        return False
    
    blue_x, blue_y = find_center(segments_matrix_blue, segments_descriptors_blue, blue_arrow)
    red_x, red_y = find_center(segments_matrix_red, segments_descriptors_red, red_arrow)
    
    blue_x = blue_x + segments_descriptors_blue[blue_arrow][0][0]
    blue_y = blue_y + segments_descriptors_blue[blue_arrow][0][2]
    
    red_x = red_x + segments_descriptors_red[red_arrow][0][0]
    red_y = red_y + segments_descriptors_red[red_arrow][0][2]
    
    blue_p1 = np.array([segments_descriptors_blue[blue_arrow][0][0], segments_descriptors_blue[blue_arrow][0][2]])
    blue_p2 = np.array([segments_descriptors_blue[blue_arrow][0][1], segments_descriptors_blue[blue_arrow][0][3]])

    red_p1 = np.array([segments_descriptors_red[red_arrow][0][0], segments_descriptors_red[red_arrow][0][2]])
    red_p2 = np.array([segments_descriptors_red[red_arrow][0][1], segments_descriptors_red[red_arrow][0][3]])
    
    blue_len = LA.norm(blue_p1 - blue_p2)
    red_len = LA.norm(red_p1 - red_p2)

    if LA.norm(np.array([blue_x, blue_y]) - np.array([red_x, red_y])) < (blue_len + red_len) * 0.37:
        return True
    else:
        return False

In [21]:
###################################################
############# Wykryj logo Carrefour ###############
###################################################

def detect_carrefour_logo(img):
    detected_blue, segments_descriptors_blue, segments_matrix_blue = detect_blue_arrow(img)
    detected_red, segments_descriptors_red, segments_matrix_red = detect_red_arrow(img)
    
    red_blue_pairs = []
    for b in detected_blue:
        for r in detected_red:
            if is_connected(b, r, segments_descriptors_blue, segments_descriptors_red, segments_matrix_blue, segments_matrix_red):
                red_blue_pairs.append((b, r))
                
    detected_logos = []
    for (b, r) in red_blue_pairs:
        min_i = min(segments_descriptors_blue[b][0][0], segments_descriptors_red[r][0][0])
        max_i = max(segments_descriptors_blue[b][0][1], segments_descriptors_red[r][0][1])
        min_j = min(segments_descriptors_blue[b][0][2], segments_descriptors_red[r][0][2])
        max_j = max(segments_descriptors_blue[b][0][3], segments_descriptors_red[r][0][3])

        detected_logos.append([min_i, max_i, min_j, max_j])
    print("Wykryto:", len(detected_logos), "logotypów.")
    result = img
    for min_i, max_i, min_j, max_j in detected_logos:
        result[[min_i, max_i], min_j:max_j] = [0, 0, 0]
        result[min_i:max_i, [min_j, max_j]] = [0, 0, 0]
        
    return result

In [22]:
###################################################
############### Skrypt wykonujący #################
###################################################

name = "1"
extention = ".jpg"

time1 = time.time()
img = cv2.imread(name+extention, 3)
detected_img = detect_carrefour_logo(img)
time2 = time.time()
print("Czas trwania: ", (time2-time1))

cv2.imshow('Result', detected_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite(name+"_result"+extention, detected_img)


Kolor niebieski
Identyfikacja segmentów                                

A Jupyter Widget


Obliczenie cech                        

A Jupyter Widget


Analiza cech

...

A Jupyter Widget


Analiza cech

A Jupyter Widget


Wykryto: 1 logotypów.
Czas trwania:  73.86029148101807


True